In [1]:
import pandas as pd
df = pd.read_csv('titanic3.csv')
df['family']=df.sibsp+df.parch
# dfo=df[['survived','pclass','sex','age','family']]
# df=df[['pclass','sex','age','family']]
def sex(x):
    if x == 'female':
        return 1
    else:
        return 0
df.sex=df.sex.apply(sex)

def age(x):
    if x<10:
        return 1
    elif x<20:
        return 2
    elif x<40:
        return 3
    elif x<50:
        return 4
    elif x>=50:
        return 5
    

df.age=df.age.apply(age)
df=df.fillna(0)

df


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,family
0,1,1,"Allen, Miss. Elisabeth Walton",1,3.0,0,0,24160,211.3375,B5,S,2,0.0,"St Louis, MO",0
1,1,1,"Allison, Master. Hudson Trevor",0,1.0,1,2,113781,151.5500,C22 C26,S,11,0.0,"Montreal, PQ / Chesterville, ON",3
2,1,0,"Allison, Miss. Helen Loraine",1,1.0,1,2,113781,151.5500,C22 C26,S,0,0.0,"Montreal, PQ / Chesterville, ON",3
3,1,0,"Allison, Mr. Hudson Joshua Creighton",0,3.0,1,2,113781,151.5500,C22 C26,S,0,135.0,"Montreal, PQ / Chesterville, ON",3
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",1,3.0,1,2,113781,151.5500,C22 C26,S,0,0.0,"Montreal, PQ / Chesterville, ON",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",1,2.0,1,0,2665,14.4542,0,C,0,328.0,0,1
1305,3,0,"Zabour, Miss. Thamine",1,0.0,1,0,2665,14.4542,0,C,0,0.0,0,1
1306,3,0,"Zakarian, Mr. Mapriededer",0,3.0,0,0,2656,7.2250,0,C,0,304.0,0,0
1307,3,0,"Zakarian, Mr. Ortin",0,3.0,0,0,2670,7.2250,0,C,0,0.0,0,0


In [2]:
from sklearn.model_selection import train_test_split
y_df=df.survived 
X_df=df[['pclass','sex','age','family','fare']]

X_train,X_test,y_train,y_test=train_test_split(X_df,y_df,test_size=0.2,random_state=10)

print(X_train.shape)
print(X_test.shape)



(1047, 5)
(262, 5)


In [4]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


rf_model=RandomForestClassifier(max_depth=6, n_estimators=1000,random_state=1)
rf_model.fit(X=X_train,y=y_train)
rf_pred=rf_model.predict(X_test)
accuracy_rf=accuracy_score(y_test, rf_pred)

lr_model=LogisticRegression()
lr_model.fit(X_train,y_train)
lr_pred=lr_model.predict(X_test)
accuracy_lr=accuracy_score(y_test,lr_pred).round(2)

accuracy_rf
round(accuracy_lr,2)

0.77

In [292]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,MinMaxScaler,StandardScaler


estimators = [('scaler', MinMaxScaler()),
             ('clf', RandomForestClassifier(random_state=1))]

pipe = Pipeline(estimators)

In [293]:
from sklearn.model_selection import GridSearchCV

params = [{
    'clf__max_depth': [6, 8, 10, 12,],
    'clf__n_estimators': [50, 100, 500,1000]
}]

gridsearch = GridSearchCV(
    estimator=pipe, param_grid=params, return_train_score=True, cv=5, verbose=2)

gridsearch.fit(X_df, y_df)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END .............clf__max_depth=6, clf__n_estimators=50; total time=   0.0s
[CV] END .............clf__max_depth=6, clf__n_estimators=50; total time=   0.0s
[CV] END .............clf__max_depth=6, clf__n_estimators=50; total time=   0.0s
[CV] END .............clf__max_depth=6, clf__n_estimators=50; total time=   0.0s
[CV] END .............clf__max_depth=6, clf__n_estimators=50; total time=   0.0s
[CV] END ............clf__max_depth=6, clf__n_estimators=100; total time=   0.0s
[CV] END ............clf__max_depth=6, clf__n_estimators=100; total time=   0.0s
[CV] END ............clf__max_depth=6, clf__n_estimators=100; total time=   0.0s
[CV] END ............clf__max_depth=6, clf__n_estimators=100; total time=   0.0s
[CV] END ............clf__max_depth=6, clf__n_estimators=100; total time=   0.0s
[CV] END ............clf__max_depth=6, clf__n_estimators=500; total time=   0.4s
[CV] END ............clf__max_depth=6, clf__n_es

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('clf',
                                        RandomForestClassifier(random_state=1))]),
             param_grid=[{'clf__max_depth': [6, 8, 10, 12],
                          'clf__n_estimators': [50, 100, 500, 1000]}],
             return_train_score=True, verbose=2)

In [294]:
pred = gridsearch.best_estimator_.predict(X_test)
print(accuracy_score(y_test, pred))

0.851145038167939
